# 切木头最大价值问题

In [1]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 33]

In [2]:
from collections import defaultdict

In [70]:
price = defaultdict(int)

In [71]:
for i, p in enumerate(original_price): 
    price[i + 1] = p

In [72]:
price # 长度与价值的对应关系

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 33})

In [66]:
# defaultdict，顾名思义，就是带default值的dict。好处：如果keyerror，不会报错而回返回定义值
price[11], price[66]

(33, 0)

## Getting the max-price spliting by enumerate

In [7]:
def r(n):
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1, n)]
    )

In [8]:
%time r(12)

Wall time: 133 ms


35

In [9]:
%time r(15)

Wall time: 4.31 s


43

**指数级算法复杂度，后面变得非常慢**

**时间复杂度的估算：**

C为常数项：

f(n) = C * f(n-1)

f(n-1) = C * f(n-2)

f(n-2) = C * f(n-3)

...

f(1) = C

连续相乘并消项可得：

f(n) = $C^n$

T(f(n)) = T(f(n-1)) + T(f(n-2)) + ... + T(f(2))

T(f(n-1)) = T(f(n-2)) + ...+ T(f(2))

...

T(f(n)) = $(n-1)^n$

### f(n) = f(n/2) + f(n/2)
--> $O(n^2)$
### f(n) = f(n-1) + f(n-2) + ... + f(1)
--> $O(n!)$
### Merge Sort
--> $O(nlogn)$

### 番外1：递归求解斐波那契数列的例子

In [10]:
"""
0 1 1 2 3 5 8 13 21 34 55

fib(0) = 0
fib(1) = 1
fib(2) = 0 + 1 = 1
fib(3) = fib(2) + fib(1) = 2
fib(4) = fib(3) + fib(2) = 2 + 1 = 3
fib(5) = fib(4) + fib(3) = 3 + 2 = 5
"""
def fib(n):
    if n <= 1: return n   # 第1，2个数就等于它们自己，n
    else:
        return fib(n-1) + fib(n-2)  # 从后面开始每一个数就等于前两个数之和

In [11]:
# 33 以后就开始有点慢了
%time fib(33)

Wall time: 1.07 s


3524578

### 番外2：一种更高效的fibonacci算法（带缓存器）

In [12]:
'''
使用cached缓存已经计算过的数
'''
cache = {}
def cached_fib(n):
    if n in cache: return cache[n]    # 如果n在缓存里，直接返回数值即可
    else:                             # 下面的话，其实和上面的算法一样，只不过，会把计算过的数放进缓存器而已
        if n <= 1:
            cache[n] = n                  # *** 缓存操作
            return n
        else:
            result = cached_fib(n-1) + cached_fib(n-2)
            cache[n] = result             # *** 缓存操作
            return result

In [13]:
%time cached_fib(2000)

Wall time: 1.99 ms


4224696333392304878706725602341482782579852840250681098010280137314308584370130707224123599639141511088446087538909603607640194711643596029271983312598737326253555802606991585915229492453904998722256795316982874482472992263901833716778060607011615497886719879858311468870876264597369086722884023654422295243347964480139515349562972087652656069529806499841977448720155612802665404554171717881930324025204312082516817125

**提速效果非常明显**

In [14]:
%%timeit
cached_fib(15)

113 ns ± 1.37 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [15]:
%%timeit
fib(15)

222 µs ± 41.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Decorator

In [16]:
import time
def complex_f(x):
    time.sleep(1)
    return x**2

In [17]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


**问题：如果要对某个函数加一个计时功能**

**方法1：**写一个函数，这个函数调用f函数，传入参数，并且可以打印时间

In [18]:
def get_time(f, *args, **kwargs):
    st = time.time()
    result = f(*args, **kwargs)
    et = time.time()
    print("time used: {} seconds".format(et-st))
    return result

In [19]:
get_time(complex_f, 20)

time used: 1.0007662773132324 seconds


400

这个方法还是不够好，因为可能还要修改源代码中用到某个函数的内容，所以还是需要修改很多，于是：

**方法2：**写一个装饰器函数，这个函数对不调用f函数，而仅仅是对其进行封装然后返回封装后增加了功能的函数，这个函数带有了打印时间的功能

In [20]:
def get_time_wrapper(f):
    '''对f函数进行一个封装，然后返回这个封装后的函数'''
    def wrap(*args, **kwargs):
        """能够获得时间"""
        st = time.time()
        result = f(*args, **kwargs)    # wrap函数实际上传入参数调用的就是f函数
        et = time.time()
        print("time used: {} seconds".format(et-st))
        return result
    return wrap

In [21]:
get_time_wrapper(complex_f)(20)

time used: 1.0004446506500244 seconds


400

In [22]:
complex_f_new = get_time_wrapper(complex_f)

In [23]:
complex_f_new(20)

time used: 1.0008854866027832 seconds


400

最后，终极形式就是：

In [24]:
@get_time_wrapper
def complex_f2(n):
    """另一个复杂的函数"""
    time.sleep(0.23)
    return n**2

In [25]:
complex_f2(6)

time used: 0.23073840141296387 seconds


36

In [26]:
help(complex_f2)

Help on function wrap in module __main__:

wrap(*args, **kwargs)
    能够获得时间



## Analysis: How to optimize
## A Simpler Problem
#### We could make Fib Problem quick very easy!

In [27]:
from functools import wraps

In [31]:
"""
写一个装饰器，用来暂存
"""
def memo(f): 
    memo.already_computed = {}  
    # python 自带的 cache 有 lru_cache 缓存管理功能，会自动删除不常用的缓存，以释放内存空间
    @wraps(f)    # 把f的性质，文档等赋予_wrap
    def _wrap(arg):
        result = None
        
        if arg in memo.already_computed:              # 如果在缓存里，就从缓存里取
            result = memo.already_computed[arg]
            # print('hit {}'.format(arg))
        else:                                         # 如果不在缓存里，就计算，然后写进缓存
            result = f(arg)
            memo.already_computed[arg] = result
        
        return result

    return _wrap

In [32]:
@memo
def r(n):
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1, n)]
    )

In [33]:
r(15)

43

In [43]:
memo.already_computed = {}

In [35]:
r(100)

300

In [36]:
@memo
def fib(n):
    if n <= 1: return n   # 第1，2个数就等于它们自己，n
    else:
        return fib(n-1) + fib(n-2)  # 从后面开始每一个数就等于前两个数之和

In [97]:
fib(200)

hit 1
hit 2
hit 3
hit 4
hit 5
hit 6
hit 7
hit 8
hit 9
hit 10
hit 11
hit 12
hit 13
hit 14
hit 15
hit 16
hit 17
hit 18
hit 19
hit 20
hit 21
hit 22
hit 23
hit 24
hit 25
hit 26
hit 27
hit 28
hit 29
hit 30
hit 31
hit 32
hit 33
hit 34
hit 35
hit 36
hit 37
hit 38
hit 39
hit 40
hit 41
hit 42
hit 43
hit 44
hit 45
hit 46
hit 47
hit 48
hit 49
hit 50
hit 51
hit 52
hit 53
hit 54
hit 55
hit 56
hit 57
hit 58
hit 59
hit 60
hit 61
hit 62
hit 63
hit 64
hit 65
hit 66
hit 67
hit 68
hit 69
hit 70
hit 71
hit 72
hit 73
hit 74
hit 75
hit 76
hit 77
hit 78
hit 79
hit 80
hit 81
hit 82
hit 83
hit 84
hit 85
hit 86
hit 87
hit 88
hit 89
hit 90
hit 91
hit 92
hit 93
hit 94
hit 95
hit 96
hit 97
hit 98
hit 99
hit 100
hit 101
hit 102
hit 103
hit 104
hit 105
hit 106
hit 107
hit 108
hit 109
hit 110
hit 111
hit 112
hit 113
hit 114
hit 115
hit 116
hit 117
hit 118
hit 119
hit 120
hit 121
hit 122
hit 123
hit 124
hit 125
hit 126
hit 127
hit 128
hit 129
hit 130
hit 131
hit 132
hit 133
hit 134
hit 135
hit 136
hit 137
hit 138
hit 

280571172992510140037611932413038677189525

### Dynamic Programming
1940s Bellman 数学家
通过把中间结果存起来，然后不断查询，加快计算速度的方法（不断查表法）

Programming：写表，查表

Dynamic Programming：动态写表查表

### 上面我们已经解决了最大价值切分木头问题，可以计算出最大价值，可是具体怎么切分呢？

简单修改一下：

In [73]:
solutions = {}

In [74]:
@memo
def r(n):
    
    notes = [(price[n], 0, n)] # 初始化一个notes
    for i in range(1, n):
        notes.append((r(i)+r(n-i), i, n-i)) # 对于每一个切分情况，计算最大价值
    
    max_price, left, right = max(notes, key=lambda x: x[0])
    
    solutions[n] = (left, right) # 由于递归，所以每一种情况都会append进去
    
    return max_price

In [75]:
memo.already_computed = {}

In [76]:
r(20)

60

In [77]:
solutions

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (0, 11),
 12: (2, 10),
 13: (2, 11),
 14: (3, 11),
 15: (2, 13),
 16: (6, 10),
 17: (6, 11),
 18: (2, 16),
 19: (2, 17),
 20: (10, 10)}

In [78]:
r(30)

90

In [79]:
solutions

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (0, 11),
 12: (2, 10),
 13: (2, 11),
 14: (3, 11),
 15: (2, 13),
 16: (6, 10),
 17: (6, 11),
 18: (2, 16),
 19: (2, 17),
 20: (10, 10),
 21: (10, 11),
 22: (11, 11),
 23: (2, 21),
 24: (2, 22),
 25: (3, 22),
 26: (6, 20),
 27: (6, 21),
 28: (6, 22),
 29: (2, 27),
 30: (10, 20)}

In [89]:
r(200)

600

Then we try to parse the solution:

In [85]:
def parse_solution(n):
    left_split, right_split = solutions[n]
    
    if left_split == 0: return [right_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [88]:
" ⊙ ".join(map(str, parse_solution(30)))

'10 ⊙ 10 ⊙ 10'

In [94]:
" ⊙ ".join(map(str, parse_solution(199)))

'10 ⊙ 10 ⊙ 10 ⊙ 10 ⊙ 10 ⊙ 10 ⊙ 10 ⊙ 10 ⊙ 10 ⊙ 10 ⊙ 11 ⊙ 11 ⊙ 11 ⊙ 11 ⊙ 11 ⊙ 11 ⊙ 11 ⊙ 11 ⊙ 11'

### --> 动态规划解决问题的3个步骤
#### 1. 识别并拆分子问题
#### 2. 去除子问题的重复
#### 3. 解析问题求解过程

# Edit Distance 编辑距离

## Edit Distance

通过**编辑操作**来衡量两段文字的距离，本质上也是**距离算法**。

### 编辑操作包括
- 插入操作 Insertion
- 删除操作 Deletion
- 替换操作 Substitution

### 应用领域
#### 1. 拼写纠错 Spell Correction
- e.g. Typed "Biejing" - Shoud be "Biejie"? or "Beijing"? or "Beijin"? or others?

#### 2. 评估/改进 机器翻译或者语音识别的结果
- e.g. Result "The salesman said  he   made a big   fortunate." -> "The salesman said he made a big fortunate."

In [96]:
from functools import lru_cache

In [99]:
solution={}

In [100]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    '''
    D(i,j) = min(D)
    '''
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation
    
    return min_distance

In [101]:
edit_distance('ABCDE', 'ABCCEF')

2

In [102]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F'}

In [103]:
edit_distance("beijing", "biejing")

2

In [104]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('b', 'b'): '',
 ('b', 'bi'): 'ADD i',
 ('b', 'bie'): 'ADD e',
 ('b', 'biej'): 'ADD j',
 ('b', 'bieji'): 'ADD i',
 ('b', 'biejin'): 'ADD n',
 ('b', 'biejing'): 'ADD g',
 ('be', 'b'): 'DEL e',
 ('be', 'bi'): 'SUB e => i'

In [105]:
edit_distance("1abcd11", "11abcd1")

2

In [106]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('b', 'b'): '',
 ('b', 'bi'): 'ADD i',
 ('b', 'bie'): 'ADD e',
 ('b', 'biej'): 'ADD j',
 ('b', 'bieji'): 'ADD i',
 ('b', 'biejin'): 'ADD n',
 ('b', 'biejing'): 'ADD g',
 ('be', 'b'): 'DEL e',
 ('be', 'bi'): 'SUB e => i'

In [110]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('b', 'b'): '',
 ('b', 'bi'): 'ADD i',
 ('b', 'bie'): 'ADD e',
 ('b', 'biej'): 'ADD j',
 ('b', 'bieji'): 'ADD i',
 ('b', 'biejin'): 'ADD n',
 ('b', 'biejing'): 'ADD g',
 ('be', 'b'): 'DEL e',
 ('be', 'bi'): 'SUB e => i'

In [111]:
edit_distance('5678', '')

4

In [112]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('b', 'b'): '',
 ('b', 'bi'): 'ADD i',
 ('b', 'bie'): 'ADD e',
 ('b', 'biej'): 'ADD j',
 ('b', 'bieji'): 'ADD i',
 ('b', 'biejin'): 'ADD n',
 ('b', 'biejing'): 'ADD g',
 ('be', 'b'): 'DEL e',
 ('be', 'bi'): 'SUB e => i'

### 课后作业1：解析编辑方法

In [117]:
def parse_editting_method(X, Y):
    '''
    分析一下：
    -1. 如果X, Y 都为空，那么返回None
    0. 如果X, Y其中有一个为 '': 那么返回一个操作，然后尝试返回parse(X[:-1], Y[:-1])
    1. 如果X，Y找到的答案是 '': 那么就尝试返回parse(X[:-1], Y[:-1])，如果报
    2. 如果X，Y找到的答案是 'SUB ...': 那么就返回替换操作，然后返回parse(X[:-1], Y[:-1])
    3. 如果X，Y找到的答案是 'ADD...' or 'DEL...'，那么就做相应操作，然后返回parse(X+操作, Y)
    '''
    if X == Y: return []
    if X == '' and Y == '': return []
    if X == '': return ["ADD {}".format(Y[-1])] + parse_editting_method(X[:-1], Y[:-1])
    if Y == '': return ["DEL {}".format(X[-1])] + parse_editting_method(X[:-1], Y[:-1])
    if solution[(X, Y)] == '': return parse_editting_method(X[:-1], Y[:-1])
    if solution[(X, Y)].split()[0] == 'SUB': return [solution[(X, Y)]] + parse_editting_method(X[:-1], Y[:-1])
    if solution[(X, Y)].split()[0] == 'ADD': return [solution[(X, Y)]] + parse_editting_method(X+solution[(X, Y)].split()[1], Y)
    if solution[(X, Y)].split()[0] == 'DEL': return [solution[(X, Y)]] + parse_editting_method(X[:-1], Y)

In [118]:
parse_editting_method('A', 'B')

['SUB A => B']

In [119]:
parse_editting_method('beijing', 'biejing')

['DEL i', 'ADD i']

In [120]:
parse_editting_method("1abcd11", "11abcd1")

['DEL 1', 'ADD 1']

In [121]:
parse_editting_method("abcd", "")

['DEL d', 'DEL c', 'DEL b', 'DEL a']

In [122]:
parse_editting_method("", "abcd")

['ADD d', 'ADD c', 'ADD b', 'ADD a']

In [130]:
parse_editting_method("", "abcd")

['ADD d', 'ADD c', 'ADD b', 'ADD a']

### 课后作业2：拼音自动纠错

In [131]:
import pinyin

In [132]:
CHN_CHAR_SET = open("article_9k.txt", encoding='utf-8').read()

In [134]:
CHN_CHAR_SET[:1000]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体验版内测稳定版暂不受影响以确保工程师可以集中全部精力进行系统优化工作有人猜测这也是将精力主要用到MIUI9的研发之中MIUI8去年5月发布距今已有一年有余也是时候更新换代了当然关于MIUI9的确切信息我们还是等待官方消息\n骁龙835作为唯一通过Windows10桌面平台认证的ARM处理器高通强调不会因为只考虑性能而去屏蔽掉小核心相反他们正联手微软找到一种适合桌面平台的兼顾性能和功耗的完美方案报道称微软已经拿到了一些新的源码以便Windows10更好地理解biglittle架构资料显示骁龙835作为一款集成了CPUGPU基带蓝牙WiFi的SoC比传统的Wintel方案可以节省至少30的PCB空间按计划今年Q4华硕惠普联想将首发骁龙835Win10电脑预计均是二合一形态的产品当然高通骁龙只是个开始未来也许还能见到三星Exynos联发科华为麒麟小米澎湃等进入Windows10桌面平台\n此前的一加3T搭载的是3400mAh电池DashCharge快充规格为5V4A至于电池缩水可能与刘作虎所说一加手机5要做市面最轻薄大屏旗舰的设定有关按照目前掌握的资料一加手机5拥有55寸1080P三星AMOLED显示屏6G8GBRAM64GB128GBROM双1600万摄像头备货量惊喜根据京东泄露的信息一加5起售价是xx99元应该是在279928992999中的某个\n这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车新华社记者张立云摄\n原标题44岁女子跑深圳约会网友被拒暴雨中裸身奔走深圳交警微博称昨日清晨交警发现有一女子赤裸上身行走在南坪快速上期间还起了轻生年头一辅警发现后赶紧为其披上黄衣并一路劝说她那么事发时到底都发生了些什么呢南都记者带您一起还原现场南都记者在龙岗大队坂田中队见到了辅警刘青发现女生的辅警一位外表高大帅气说话略带些腼腆的90后青年刘青介绍6月16日早上7时36分他正在环城南路附近值勤接到中队关于一位女子裸身进入机动车可能有危险的警情随后骑着小铁骑开始沿路寻找大概花了十多分钟在南坪大道坂田出口往龙岗方向的逆行辅道上发现该女子女子身上一丝不挂地逆车流而行时走时停时坐时躺险象环生刘青停好小铁骑和另外一名巡防员追了上去发现女子的情绪很低落话不多刘青尝试和女子交流劝说女子离开可女

In [139]:
def chn_to_py(character):
    return pinyin.get(character, format='strip', delimiter=' ')

In [140]:
chn_to_py("我是谁")

'wo shi shui'

In [141]:
CHN_PY_SET = chn_to_py(CHN_CHAR_SET)

In [142]:
len(CHN_PY_SET)

129433034

In [144]:
import re
def token(text):
    return re.findall('[a-z]+', text.lower())

In [145]:
token(CHN_PY_SET[:100])

['ci',
 'wai',
 'zi',
 'ben',
 'zhou',
 'yue',
 'ri',
 'qi',
 'chu',
 'xiao',
 'mi',
 'shou',
 'ji',
 'deng',
 'kuan',
 'ji',
 'xing',
 'wai',
 'qi',
 'yu',
 'ji',
 'xing',
 'yi']

In [146]:
from collections import Counter, defaultdict

In [147]:
PINYIN_COUNT = Counter(token(CHN_PY_SET))

In [148]:
PINYIN_COUNT

Counter({'ci': 95927,
         'wai': 131353,
         'zi': 192620,
         'ben': 55466,
         'zhou': 80326,
         'yue': 258519,
         'ri': 278379,
         'qi': 251164,
         'chu': 156005,
         'xiao': 126736,
         'mi': 40857,
         'shou': 175635,
         'ji': 645276,
         'deng': 77672,
         'kuan': 13631,
         'xing': 241741,
         'yu': 302949,
         'yi': 682478,
         'zan': 6671,
         'ting': 31732,
         'geng': 35528,
         'xin': 359619,
         'fa': 233137,
         'bu': 281533,
         'han': 39849,
         'kai': 94310,
         'ban': 83215,
         'ti': 176853,
         'yan': 125085,
         'nei': 55595,
         'ce': 30897,
         'wen': 121636,
         'ding': 60586,
         'ying': 140068,
         'xiang': 196711,
         'que': 26243,
         'bao': 132256,
         'gong': 259593,
         'cheng': 210265,
         'shi': 860634,
         'ke': 173705,
         'zhong': 409418,
     

In [191]:
def correct_pinyin(word):
    candidates = (known(edits0(word)) or 
                 known(edits1(word)) or
                 known(edits2(word)) or
                 [word])
                
    return max(candidates, key=lambda x: PINYIN_COUNT.get(x))

In [178]:
def known(words):
    '''
    判断某个近词是否在list中
    '''
    return [w for w in words if w in PINYIN_COUNT]
    
def edits0(word):
    return [word]

def edits2(word):
    '''编辑距离为2的词，就是所有那些与目标词编辑距离为1的词 编辑距离为1的词'''
    return [e2 for e1 in edits1(word) for e2 in edits1(e1)]

In [161]:
def edits1(word):
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)
    
def splits(word):
    '''Return a list of all possible (first, rest) pairs that comprise pinyin'''
    return [(word[:i], word[i:])
             for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [162]:
splits('zhongwen')

[('', 'zhongwen'),
 ('z', 'hongwen'),
 ('zh', 'ongwen'),
 ('zho', 'ngwen'),
 ('zhon', 'gwen'),
 ('zhong', 'wen'),
 ('zhongw', 'en'),
 ('zhongwe', 'n'),
 ('zhongwen', '')]

In [163]:
print(edits0('zhongguo'))

['zhongguo']


In [164]:
print(edits1('zhongguo'))

{'zhongouo', 'zhonggur', 'zhonggqo', 'zhsongguo', 'gzhongguo', 'mhongguo', 'zbhongguo', 'zhongguoy', 'zhomgguo', 'zhopgguo', 'zhonmgguo', 'zhongguo', 'zhonggto', 'zhonghuo', 'zhonggquo', 'zkhongguo', 'zhonggvo', 'zhonrguo', 'zhoigguo', 'zhongjguo', 'jzhongguo', 'zhongmguo', 'zhongduo', 'znongguo', 'whongguo', 'zhonggun', 'zhongguof', 'ztongguo', 'zhontguo', 'zhosngguo', 'zhlngguo', 'zhongsuo', 'zhoniguo', 'zhonggul', 'zhonmguo', 'zhongguoc', 'ihongguo', 'zhongwuo', 'xzhongguo', 'xhongguo', 'zhongguu', 'pzhongguo', 'shongguo', 'zjongguo', 'zhongguqo', 'dhongguo', 'zzhongguo', 'yzhongguo', 'zhonggupo', 'zhongyuo', 'zhongguxo', 'zihongguo', 'azhongguo', 'zhongugo', 'zhmngguo', 'zhoxgguo', 'zhonwgguo', 'hhongguo', 'zhkongguo', 'zphongguo', 'zhongxguo', 'zhonygguo', 'zhgongguo', 'zhonggko', 'zhdongguo', 'zhoangguo', 'zhongluo', 'zhongguq', 'zgongguo', 'zhofgguo', 'zhongqguo', 'zhongjuo', 'zhobngguo', 'zhlongguo', 'zhonggumo', 'zhongguuo', 'jhongguo', 'zhonggut', 'zbongguo', 'zhowngguo', 'zh

In [171]:
def get_deletes(word):
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    return deletes

def get_transposes(word):
    pairs      = splits(word)
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    return transposes

def get_replaces(word):
    pairs      = splits(word)
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    return replaces
    
def get_inserts(word):
    pairs      = splits(word)
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return inserts

In [170]:
print(get_deletes('zhongguo'))

['hongguo', 'zongguo', 'zhngguo', 'zhogguo', 'zhonguo', 'zhonguo', 'zhonggo', 'zhonggu']


In [172]:
print(get_transposes('zhongguo'))

['hzongguo', 'zohngguo', 'zhnogguo', 'zhognguo', 'zhongguo', 'zhongugo', 'zhonggou']


In [173]:
print(get_replaces('zhongguo'))

['ahongguo', 'bhongguo', 'chongguo', 'dhongguo', 'ehongguo', 'fhongguo', 'ghongguo', 'hhongguo', 'ihongguo', 'jhongguo', 'khongguo', 'lhongguo', 'mhongguo', 'nhongguo', 'ohongguo', 'phongguo', 'qhongguo', 'rhongguo', 'shongguo', 'thongguo', 'uhongguo', 'vhongguo', 'whongguo', 'xhongguo', 'yhongguo', 'zhongguo', 'zaongguo', 'zbongguo', 'zcongguo', 'zdongguo', 'zeongguo', 'zfongguo', 'zgongguo', 'zhongguo', 'ziongguo', 'zjongguo', 'zkongguo', 'zlongguo', 'zmongguo', 'znongguo', 'zoongguo', 'zpongguo', 'zqongguo', 'zrongguo', 'zsongguo', 'ztongguo', 'zuongguo', 'zvongguo', 'zwongguo', 'zxongguo', 'zyongguo', 'zzongguo', 'zhangguo', 'zhbngguo', 'zhcngguo', 'zhdngguo', 'zhengguo', 'zhfngguo', 'zhgngguo', 'zhhngguo', 'zhingguo', 'zhjngguo', 'zhkngguo', 'zhlngguo', 'zhmngguo', 'zhnngguo', 'zhongguo', 'zhpngguo', 'zhqngguo', 'zhrngguo', 'zhsngguo', 'zhtngguo', 'zhungguo', 'zhvngguo', 'zhwngguo', 'zhxngguo', 'zhyngguo', 'zhzngguo', 'zhoagguo', 'zhobgguo', 'zhocgguo', 'zhodgguo', 'zhoegguo', 'zh

In [176]:
print(get_inserts('zhongguo'))

['azhongguo', 'bzhongguo', 'czhongguo', 'dzhongguo', 'ezhongguo', 'fzhongguo', 'gzhongguo', 'hzhongguo', 'izhongguo', 'jzhongguo', 'kzhongguo', 'lzhongguo', 'mzhongguo', 'nzhongguo', 'ozhongguo', 'pzhongguo', 'qzhongguo', 'rzhongguo', 'szhongguo', 'tzhongguo', 'uzhongguo', 'vzhongguo', 'wzhongguo', 'xzhongguo', 'yzhongguo', 'zzhongguo', 'zahongguo', 'zbhongguo', 'zchongguo', 'zdhongguo', 'zehongguo', 'zfhongguo', 'zghongguo', 'zhhongguo', 'zihongguo', 'zjhongguo', 'zkhongguo', 'zlhongguo', 'zmhongguo', 'znhongguo', 'zohongguo', 'zphongguo', 'zqhongguo', 'zrhongguo', 'zshongguo', 'zthongguo', 'zuhongguo', 'zvhongguo', 'zwhongguo', 'zxhongguo', 'zyhongguo', 'zzhongguo', 'zhaongguo', 'zhbongguo', 'zhcongguo', 'zhdongguo', 'zheongguo', 'zhfongguo', 'zhgongguo', 'zhhongguo', 'zhiongguo', 'zhjongguo', 'zhkongguo', 'zhlongguo', 'zhmongguo', 'zhnongguo', 'zhoongguo', 'zhpongguo', 'zhqongguo', 'zhrongguo', 'zhsongguo', 'zhtongguo', 'zhuongguo', 'zhvongguo', 'zhwongguo', 'zhxongguo', 'zhyongguo'

In [187]:
known(get_inserts('pin'))

['pian', 'ping']

In [194]:
correct_pinyin('piny')

'ping'

In [199]:
correct_pinyin('zhogn')

'zhong'

In [201]:
correct_pinyin("yinn")

'ying'

**写一个函数，能够改正一个序列**

In [203]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct_pinyin, text_pinyin.split()))

In [204]:
correct_sequence_pinyin("wo xiagn shnag qinng hau da xue")

'wo xian shang qing hua da xue'

**思考题："woxiagnshnagqinnghaudaxue" --》 "wo xiagn shnag qinng hau da xue"**

使用1-gram，2-gram建立语料库，然后，找到最大概率切分方式：

w oxiagnshnagqinnghaudaxue

wo xiagnshnagqinnghaudaxue

wox iagnshnagqinnghaudaxue

...

wo xiagn shnagqinnghaudaxue

**get_max_split_probability**